In [52]:
import numpy as np
import nltk
import string
import random

In [53]:
f=open('chatbot.txt','r', errors ='ignore')
raw_doc=f.read()
raw_doc=raw_doc.lower()
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_doc)
word_tokens= nltk.word_tokenize(raw_doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [54]:
sent_tokens[:2]

['loan\nfrom wikipedia, the free encyclopedia\njump to navigationjump to search\nfor other uses, see loan (disambiguation).',
 'bank\nwhat is bank\nloan document issued by the bank of petrevene, bulgaria, dated 1936.\npart of a series on\nfinance\nphilippine-stock-market-board.jpg\nmarkets\ninstruments\ncorporate\npersonal\npublic\nbanking\nregulation · financial law\neconomic history\nemblem-money.svg business and economics portalbills and coins.svg money portal\nvte\npart of a series on financial services\nbanking\ntypes of banks\naccounts · cards\nfunds transfer\nterms\nautomated teller machinebank regulationloanmobile bankingmoney creation\nbank secrecyethical bankingfractional-reserve bankingfull-reserve bankingislamic bankingprivate banking\nrelated topics\n category commons portal\nvte\nin finance, a loan is the lending of money by one or more individuals, organizations, or other entities to other individuals, organizations etc.']

In [55]:
word_tokens[:2]

['loan', 'from']

In [56]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
  remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
  def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translation(remove_punct_dict)))

In [57]:
 GREET_INPUTS = ("hello", "hi", "greetings", "sup","what's up", "hey")
 GREET_RESPONSES = ["hi", "hey","*nods*" "hi there", "hello", "I am glad ! You are talking to me"]
 def greet(sentence):
   for word in sentence.split():
     if word.lower() in GREET_INPUTS:
       return random.choice(GREET_RESPONSES)

In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
def response(user_response):
  robo1_response=''
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words ='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx=vals.argsort()[0][-2] 
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf==0):
    robo1_response=robo1_response+"I am sorry! I don't understand you"
    return robo1_response
  else:
      robo1_response = robo1_response+sent_token[idx]
      return robo1_response

In [60]:
flag=True
print("BOT: My name is Mudrika. Let's have a conversation! Also, if you want to exit anytime, just type Bye!")
while(flag==True):
  user_response = input()
  user_response= user_response.lower()
  if(user_response !='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("BOT: You are welcome.")
    else:
        if(greet(user_response)!=None):
          print("BOT: "+greet(user_response))
        else:
            sent_tokens.append(user_response)
            word_tokens=word_tokens+nltk.word_tokenize(user_response)
            final_words=list(set(word_tokens))
            print("BOT: ", end="")
            print(response(user_response))
            sent_tokens.remove(user_response)
  else:
          flag=False
          print("BOT: Goodbye! Take care <3 ")

BOT: My name is Mudrika. Let's have a conversation! Also, if you want to exit anytime, just type Bye!
hey
BOT: I am glad ! You are talking to me
bank
BOT: 

NameError: ignored